# Lab session #2 - ST7-83

# This is to hand out, will count as Continuous Assessment

## To hand out

1. A **report** with your answers in a **PDF FILE** (made out of LaTeX, libreoffice, ...)
  * Math and text answers
  * The code for the circuits
  * Screenshot of figures/circuits
  * python answers and results of runs
  * *etc*
  
2. **This notebook** or **a python file**
  * as a runnable script

But first, some libraries to load (nothing to modify here)

In [41]:
! python -m pip install qiskit qiskit-aer


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import numpy as np
from math import pi, gcd
from qiskit import *
from qiskit_aer import AerSimulator, StatevectorSimulator

def processOneState(st): # Longueur = puissance de 2
        s = list(st)
        if len(s) == 2:
            return {'0' : s[0], '1' : s[1]}
        else:
            a0 = processOneState(s[:len(s)//2])
            a1 = processOneState(s[len(s)//2:])
            r = {}
            for k in a0:
                r['0' + k] = a0[k]
            for k in a1:
                r['1' + k] = a1[k]
            return r

def printOneState(d): # get a dict as per processStates output
    for k in d:
        im = d[k].imag
        re = d[k].real
        if abs(im) >= 0.001 or abs(re) >= 0.001:
            print("% .3f + % .3fj |%s>" % (re,im,k))

def printFinalRes(result):
    printOneState(processOneState(list(np.asarray(result))))


def runStateVector(qc):
    simulator = StatevectorSimulator()
    job = simulator.run(qc.decompose(reps=6), memory=True)
    job_result = job.result()
    result = job_result.results[0].to_dict()['data']['statevector']
    printFinalRes(result)

def runStateVectorSeveralTimes(qc, howmany):
    qc.save_statevector(label = 'collect', pershot = True)
    simulator = StatevectorSimulator()
    job = simulator.run(qc.decompose(reps=6), memory=True, shots=howmany)
    result = job.result()
    memory = result.data(0)['memory']
    collect = result.data(0)['collect']
    r = {}
    for i in range(len(collect)):
        r[str(collect[i])] = (0, collect[i])
    for i in range(len(collect)):
        n, v = r[str(collect[i])]
        r[str(collect[i])] = (n+1, v)
    for k in r:
        i, v = r[k]
        print(f"With {i} occurences:")
        printFinalRes(v)

def runSample(qc,howmany):
    simulator = AerSimulator()
    job = simulator.run(qc.decompose(reps=6), shots=howmany)
    res = dict(job.result().get_counts(qc))
    return res

## 1 - Grover

In this exercice, we are going to experimentally verify the amplitude amplification coming from Grover algorithm.
We shall test the algorithm on a function over `N` variables:
$$f(x_1,\ldots,x_n) = (x_1\wedge \ldots \wedge x_n)$$
The set $f^{-1}\{1\}$ is of size $1$.

## Questions

### Q 1.1 Build the circuit

Complete the missing bits in the following code. You might find useful the gate `qc.mcx(controls, target)`. For instance
```
qc.mcx(q[1:], q[0])
```
place a multi-controlled NOT gate on `q[0]` with all the other wires of `q` as controls.

In [94]:
N = 8# Number of variables of f

def oracle(qc,q,aux):
    qc.h(aux[0])
    qc.mcx(q[:], aux[0])  # Multi-controlled X using q as control and aux as target
    qc.h(aux[0])
def u0bot(qc,q):
    qc.h(q)
    qc.x(q)
    qc.h(q[-1])
    qc.mcx(q[:-1],q[-1])
    qc.h(q[-1])
    qc.x(q)
    qc.h(q)
def grover(n_iter):
    q = QuantumRegister(N)    # The variables in superposition
    c = ClassicalRegister(N)  # Registers to store their measure
    aux = QuantumRegister(1)  # to store the additional wire used for O. Note that there is no need to measure it !
    qc = QuantumCircuit(q,aux,c)

    # TODO initialize the circuit
    qc.h(q) 
    qc.x(aux[0])
    qc.h(aux[0])
    for _ in range(n_iter):
        qc.barrier()       # To physically separate each iteration (does nothing but renders the circuit more legible)
        oracle(qc,q,aux)
        qc.barrier() 
        u0bot(qc,q)

    qc.measure(q,c)        # Measure
    return qc

print(grover(2))   # Show the circuit for 2 iterations

        ┌───┐      ░                 ░ ┌───┐┌───┐          ┌───┐┌───┐      ░ »
q165_0: ┤ H ├──────░────────■────────░─┤ H ├┤ X ├───────■──┤ X ├┤ H ├──────░─»
        ├───┤      ░        │        ░ ├───┤├───┤       │  ├───┤├───┤      ░ »
q165_1: ┤ H ├──────░────────■────────░─┤ H ├┤ X ├───────■──┤ X ├┤ H ├──────░─»
        ├───┤      ░        │        ░ ├───┤├───┤       │  ├───┤├───┤      ░ »
q165_2: ┤ H ├──────░────────■────────░─┤ H ├┤ X ├───────■──┤ X ├┤ H ├──────░─»
        ├───┤      ░        │        ░ ├───┤├───┤       │  ├───┤├───┤      ░ »
q165_3: ┤ H ├──────░────────■────────░─┤ H ├┤ X ├───────■──┤ X ├┤ H ├──────░─»
        ├───┤      ░        │        ░ ├───┤├───┤       │  ├───┤├───┤      ░ »
q165_4: ┤ H ├──────░────────■────────░─┤ H ├┤ X ├───────■──┤ X ├┤ H ├──────░─»
        ├───┤      ░        │        ░ ├───┤├───┤       │  ├───┤├───┤      ░ »
q165_5: ┤ H ├──────░────────■────────░─┤ H ├┤ X ├───────■──┤ X ├┤ H ├──────░─»
        ├───┤      ░        │        ░ ├───┤├───┤   

### Q 1.2 - Run the circuit

What does the following code do ?

In [95]:
for i in range(15):
    s = runSample(grover(i),1000)
    k=""
    for _ in range(N):
        k += "1"
    if k in s:
        print(i, s[k])
    else:
        print(i, 0)

0 4
1 20
2 56
3 76
4 143
5 205
6 280
7 312
8 383
9 442
10 502
11 500
12 523
13 514
14 465


### Q 1.3 Discussion

- In you report, explain the result. Is there any regularity ? Why ? When is the maximum value obtained ? Why ? 
- Change the value of `N` to `8` and explain the changes.

## 2 - QPE

We've seen the QPE algorithm in the course, and you checked it worked with 3 qubits. Here we are going to implement it with the following unitary:

In [55]:
from qiskit.circuit.library import UnitaryGate
from qiskit.quantum_info import Operator
U = UnitaryGate(
    Operator([[1,0,0,0],
              [0,1,0,0],
              [0,0,1,0],
              [0,0,0,np.exp(pi*2j*(5/8))]]), label="U")

## Questions 

###  Q 2.1 Math questions

* What is doing this operator ? (`2j` is in Python the complex number $2\cdot i$)
* On how many qubits does it act ?
* What are its eigenvalues/eigenvectors ?
* For each eigenvector, what should QPE return with 3 bits of precisions, as seen in the course ?

#### Give answers and explanations in the separate report 

### Q 2.2 Implementing QPE

Below a template to fill in for
- realize QPE with 3 bits of precision.
- on the eigenvector of non-trivial eigenvalue

We initialized a quantum circuit with 3 registers:
 - `eig` for storing the eigenvalues
 - `phi` for storing the eigenvector
 - `ceig` for storing the result of the measurement of the eigenvalue-register.
 
Note that we only need to measure the eigenvalues!

What you will need:
 - `QFT(size)` build for you a QFT on `size` qubits.
 - `U.control()` for controlling a gate `U`. The control qubit should be placed first in the list of wires.
 - `U.inverse()` for the inverse of the gate `U`.
 - `U.power(p)` add `p` times `U` on the circuit.
 - `qc.append(U, list_of_qubits)` applies the gate `U` on the list of qubits.
 - Beware : `phi` (for instance) is not a list but a register. So if you want to concatenate it with something else, you first have to make a list out of it with `list(phi)`.

In [57]:
from qiskit.circuit.library import QFT
size_eig = 5
size_phi = 2

eig = QuantumRegister(size_eig, name="eig")
phi = QuantumRegister(size_phi, name="phi")
ceig = ClassicalRegister(size_eig, name="ceig")
qc = QuantumCircuit(eig,phi,ceig)

# To fill in
qc.x(phi)  # Apply X gates to both qubits because |11> is the eigenvector we want to analyse
qc.h(eig)

# C-U, C-U^2, C-U^4
for i in range(size_eig):
    qc.append(U.power(2**i).control(1), [eig[i]] + list(phi))

qc.append(QFT(size_eig, inverse=True), eig)

qc.measure(eig,ceig)

qc.draw()

# First, make sure that the drawing is OK.

┌───┐                                                        »
 eig_0: ┤ H ├──────■─────────────────────────────────────────────────»
        ├───┤      │                                                 »
 eig_1: ┤ H ├──────┼─────────────■───────────────────────────────────»
        ├───┤      │             │                                   »
 eig_2: ┤ H ├──────┼─────────────┼─────────────■─────────────────────»
        ├───┤      │             │             │                     »
 eig_3: ┤ H ├──────┼─────────────┼─────────────┼─────────────■───────»
        ├───┤      │             │             │             │       »
 eig_4: ┤ H ├──────┼─────────────┼─────────────┼─────────────┼───────»
        ├───┤┌─────┴──────┐┌─────┴──────┐┌─────┴──────┐┌─────┴──────┐»
 phi_0: ┤ X ├┤0           ├┤0           ├┤0           ├┤0           ├»
        ├───┤│  unitary^1 ││  unitary^2 ││  unitary^4 ││  unitary^8 │»
 phi_1: ┤ X ├┤1           ├┤1           ├┤1           ├┤1           ├»
        └───┘└────────────┘└────────────┘└────────────┘└────────────┘»
ceig: 5/═════════════════════════════════════════════════════════════»
                                                                     »
«                       ┌───────┐┌─┐            
« eig_0: ───────────────┤0      ├┤M├────────────
«                       │       │└╥┘┌─┐         
« eig_1: ───────────────┤1      ├─╫─┤M├─────────
«                       │       │ ║ └╥┘┌─┐      
« eig_2: ───────────────┤2 IQFT ├─╫──╫─┤M├──────
«                       │       │ ║  ║ └╥┘┌─┐   
« eig_3: ───────────────┤3      ├─╫──╫──╫─┤M├───
«                       │       │ ║  ║  ║ └╥┘┌─┐
« eig_4: ───────■───────┤4      ├─╫──╫──╫──╫─┤M├
«        ┌──────┴──────┐└───────┘ ║  ║  ║  ║ └╥┘
« phi_0: ┤0            ├──────────╫──╫──╫──╫──╫─
«        │  unitary^16 │          ║  ║  ║  ║  ║ 
« phi_1: ┤1            ├──────────╫──╫──╫──╫──╫─
«        └─────────────┘          ║  ║  ║  ║  ║ 
«ceig: 5/═════════════════════════╩══╩══╩══╩══╩═
«                                 0  1  2  3  4

In [58]:
# Then run the backend !

runSample(qc, 1000)

{'10100': 1000}

### Q 2.3 Exact result

- (a) Explain how it matches the expected result.
- (b) Change the $\frac68$ of the phase of $U$: use $\frac18$, then $\frac28$... What should you see? Do you see it? Give the output you see in your report. 
- (c) Change the precision : use $4$ qubits, then $5$ qubits for `eig`, and change the fraction in the phase of $U$ to $\frac{10}{16}$ : how is your code behaving?

#### Give answers and explanations in the separate report 

### Q 2.4 Approximate result

Use  $\frac13$ in the phase of $U$:
- With 3 bits of precision
- With 4 bits of precision
- With 5 bits of precision

**Question** What do you observe? Can you explain it? What do you read?

#### Give answers and explanations in the separate report 

In [ ]:
#Changing bit precision
bit_precision = 4

# Changing phase of U
phase_U = 1/3

def generate_circuit(phase_U, bit_precision):
    U = UnitaryGate(
        Operator([[1,0,0,0],
                [0,1,0,0],
                [0,0,1,0],
                [0,0,0,np.exp(pi*2j*(phase_U))]]), label="U")

    from qiskit.circuit.library import QFT
    size_eig = bit_precision
    size_phi = 2

    eig = QuantumRegister(size_eig, name="eig")
    phi = QuantumRegister(size_phi, name="phi")
    ceig = ClassicalRegister(size_eig, name="ceig")
    qc = QuantumCircuit(eig,phi,ceig)

    # To fill in
    qc.x(phi)  # Apply X gates to both qubits because |11> is the eigenvector we want to analyse
    qc.h(eig)

    # C-U, C-U^2, C-U^4
    for i in range(size_eig):
        qc.append(U.power(2**i).control(1), [eig[i]] + list(phi))

    qc.append(QFT(size_eig, inverse=True), eig)

    qc.measure(eig,ceig)

    
    return qc

    # First, make sure that the drawing is OK.

qc = generate_circuit(phase_U, bit_precision)
qc.draw()

┌───┐                                                        ┌───────┐»
 eig_0: ┤ H ├──────■─────────────────────────────────────────────────┤0      ├»
        ├───┤      │                                                 │       │»
 eig_1: ┤ H ├──────┼─────────────■───────────────────────────────────┤1      ├»
        ├───┤      │             │                                   │  IQFT │»
 eig_2: ┤ H ├──────┼─────────────┼─────────────■─────────────────────┤2      ├»
        ├───┤      │             │             │                     │       │»
 eig_3: ┤ H ├──────┼─────────────┼─────────────┼─────────────■───────┤3      ├»
        ├───┤┌─────┴──────┐┌─────┴──────┐┌─────┴──────┐┌─────┴──────┐└───────┘»
 phi_0: ┤ X ├┤0           ├┤0           ├┤0           ├┤0           ├─────────»
        ├───┤│  unitary^1 ││  unitary^2 ││  unitary^4 ││  unitary^8 │         »
 phi_1: ┤ X ├┤1           ├┤1           ├┤1           ├┤1           ├─────────»
        └───┘└────────────┘└────────────┘└────────────┘└────────────┘         »
ceig: 4/══════════════════════════════════════════════════════════════════════»
                                                                              »
«        ┌─┐         
« eig_0: ┤M├─────────
«        └╥┘┌─┐      
« eig_1: ─╫─┤M├──────
«         ║ └╥┘┌─┐   
« eig_2: ─╫──╫─┤M├───
«         ║  ║ └╥┘┌─┐
« eig_3: ─╫──╫──╫─┤M├
«         ║  ║  ║ └╥┘
« phi_0: ─╫──╫──╫──╫─
«         ║  ║  ║  ║ 
« phi_1: ─╫──╫──╫──╫─
«         ║  ║  ║  ║ 
«ceig: 4/═╩══╩══╩══╩═
«         0  1  2  3

In [ ]:
#Changing bit precision
bit_precision = 4

# Changing phase of U
phase_U = 1/3

def generate_circuit(phase_U, bit_precision):
    U = UnitaryGate(
        Operator([[1,0,0,0],
                [0,1,0,0],
                [0,0,1,0],
                [0,0,0,np.exp(pi*2j*(phase_U))]]), label="U")

    from qiskit.circuit.library import QFT
    size_eig = bit_precision
    size_phi = 2

    eig = QuantumRegister(size_eig, name="eig")
    phi = QuantumRegister(size_phi, name="phi")
    ceig = ClassicalRegister(size_eig, name="ceig")
    qc = QuantumCircuit(eig,phi,ceig)

    # To fill in
    qc.x(phi)  # Apply X gates to both qubits because |11> is the eigenvector we want to analyse
    qc.h(eig)

    # C-U, C-U^2, C-U^4
    for i in range(size_eig):
        qc.append(U.power(2**i).control(1), [eig[i]] + list(phi))

    qc.append(QFT(size_eig, inverse=True), eig)

    qc.measure(eig,ceig)

    
    return qc

    # First, make sure that the drawing is OK.

qc = generate_circuit(phase_U, bit_precision)
qc.draw()

In [40]:
# Then run the backend !

runSample(qc, 1000)

{'1011': 3,
 '1010': 2,
 '0000': 3,
 '1001': 12,
 '1111': 4,
 '1101': 6,
 '0010': 7,
 '0011': 9,
 '0001': 9,
 '0110': 168,
 '1100': 3,
 '0100': 38,
 '0111': 20,
 '1110': 5,
 '1000': 14,
 '0101': 697}

### Q 2.5 Superposition

We saw that the circuit of QPE has no problem with a superposition of eigenvectors. Try to change the initialization of `phi` with 
$$
\frac1{\sqrt2}(|\phi_1\rangle + |\phi_2\rangle),
$$
two eigenvectors of $U$ (one with trivial eigenvalue, the other one non-trivial).

Also measure the register `phi` at the end of the circuit, and analyze the result: can you explain what you see?

Try this experiment with phase $\frac38$ and $\frac13$.

#### Give answers and explanations in the separate report 

In [89]:


N = 5  # Number of precision qubits

def generate_circuit2(phase_U, bit_precision):
    """ Quantum Phase Estimation circuit with explicit measurement of phi """
    
    U = UnitaryGate(
        Operator([[1,0,0,0],
                [0,1,0,0],
                [0,0,1,0],
                [0,0,0,np.exp(np.pi * 2j * phase_U)]]), label="U")

    size_eig = bit_precision  # Eigenvalue precision qubits
    size_phi = 2  # Eigenstate qubits

    eig = QuantumRegister(size_eig, name="eig")  # Precision register
    phi = QuantumRegister(size_phi, name="phi")  # Eigenstate register
    ceig = ClassicalRegister(size_eig, name="ceig")  # Classical bits for eigenvalue
    cphi = ClassicalRegister(size_phi, name="cphi")  # Classical bits for phi measurement

    qc = QuantumCircuit(eig, phi, ceig, cphi)

    # Initialize phi as a superposition of eigenvectors
    qc.h(phi[0])       # Create superposition (|0> + |1>) / sqrt(2)
    qc.cx(phi[0], phi[1])  # Entangle to get (|00> + |11>) / sqrt(2)

    qc.h(eig)  # Apply Hadamard to eigenvalue register

    # Apply Controlled-U, Controlled-U^2, Controlled-U^4, etc.
    for i in range(size_eig):
        qc.append(U.power(2**i).control(1), [eig[i]] + list(phi))

    # Apply Inverse Quantum Fourier Transform
    qc.append(QFT(size_eig, inverse=True), eig)

    # Measurement of eig and phi registers
    qc.measure(eig, ceig)  # Measure eigenvalue qubits
    qc.measure(phi, cphi)  # Measure eigenstate qubits
    
    return qc


# Changing bit precision
bit_precision = 6

# Changing phase of U
phase_U = 1/3

qc = generate_circuit2(phase_U, bit_precision)
qc.draw()

┌───┐                                                             »
 eig_0: ┤ H ├───────────■─────────────────────────────────────────────────»
        ├───┤           │                                                 »
 eig_1: ┤ H ├───────────┼─────────────■───────────────────────────────────»
        ├───┤           │             │                                   »
 eig_2: ┤ H ├───────────┼─────────────┼─────────────■─────────────────────»
        ├───┤           │             │             │                     »
 eig_3: ┤ H ├───────────┼─────────────┼─────────────┼─────────────■───────»
        ├───┤           │             │             │             │       »
 eig_4: ┤ H ├───────────┼─────────────┼─────────────┼─────────────┼───────»
        ├───┤           │             │             │             │       »
 eig_5: ┤ H ├───────────┼─────────────┼─────────────┼─────────────┼───────»
        ├───┤     ┌─────┴──────┐┌─────┴──────┐┌─────┴──────┐┌─────┴──────┐»
 phi_0: ┤ H ├──■──┤0           ├┤0           ├┤0           ├┤0           ├»
        └───┘┌─┴─┐│  unitary^1 ││  unitary^2 ││  unitary^4 ││  unitary^8 │»
 phi_1: ─────┤ X ├┤1           ├┤1           ├┤1           ├┤1           ├»
             └───┘└────────────┘└────────────┘└────────────┘└────────────┘»
ceig: 6/══════════════════════════════════════════════════════════════════»
                                                                          »
cphi: 2/══════════════════════════════════════════════════════════════════»
                                                                          »
«                                      ┌───────┐   ┌─┐               
« eig_0: ──────────────────────────────┤0      ├───┤M├───────────────
«                                      │       │   └╥┘┌─┐            
« eig_1: ──────────────────────────────┤1      ├────╫─┤M├────────────
«                                      │       │    ║ └╥┘┌─┐         
« eig_2: ──────────────────────────────┤2      ├────╫──╫─┤M├─────────
«                                      │  IQFT │    ║  ║ └╥┘┌─┐      
« eig_3: ──────────────────────────────┤3      ├────╫──╫──╫─┤M├──────
«                                      │       │    ║  ║  ║ └╥┘┌─┐   
« eig_4: ───────■──────────────────────┤4      ├────╫──╫──╫──╫─┤M├───
«               │                      │       │    ║  ║  ║  ║ └╥┘┌─┐
« eig_5: ───────┼──────────────■───────┤5      ├────╫──╫──╫──╫──╫─┤M├
«        ┌──────┴──────┐┌──────┴──────┐└──┬─┬──┘    ║  ║  ║  ║  ║ └╥┘
« phi_0: ┤0            ├┤0            ├───┤M├───────╫──╫──╫──╫──╫──╫─
«        │  unitary^16 ││  unitary^32 │   └╥┘   ┌─┐ ║  ║  ║  ║  ║  ║ 
« phi_1: ┤1            ├┤1            ├────╫────┤M├─╫──╫──╫──╫──╫──╫─
«        └─────────────┘└─────────────┘    ║    └╥┘ ║  ║  ║  ║  ║  ║ 
«ceig: 6/══════════════════════════════════╬═════╬══╩══╩══╩══╩══╩══╩═
«                                          ║     ║  0  1  2  3  4  5 
«cphi: 2/══════════════════════════════════╩═════╩═══════════════════
«                                          0     1

In [93]:
# Then run the backend !

runSample(qc, 2000)

{'11 100000': 1,
 '11 001100': 1,
 '11 000000': 1,
 '11 011110': 1,
 '11 011111': 1,
 '11 110100': 1,
 '11 001000': 1,
 '11 010101': 710,
 '11 011010': 3,
 '11 100001': 2,
 '11 101001': 1,
 '11 001010': 1,
 '11 111101': 1,
 '11 010011': 13,
 '11 001111': 3,
 '11 000111': 1,
 '11 011001': 2,
 '11 011100': 2,
 '11 001101': 4,
 '11 011101': 1,
 '11 010000': 1,
 '11 010010': 6,
 '11 010001': 4,
 '11 110111': 1,
 '11 010100': 69,
 '11 100101': 2,
 '11 000011': 1,
 '11 010111': 20,
 '11 011011': 6,
 '11 010110': 185,
 '11 011000': 11,
 '00 000000': 943}